### 1. Import

In [1]:
import os
import json

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

from multimodal_rag import MultimodalRAG
from multimodal_embedder import create_embedder
from pdf_to_qa import generate_qa_for_pdf
from evaluation import evaluate_generation, compute_mrr_at_k


### 2. Configuration

In [2]:
PDF_FILE = "knowledge/subset_riksbanken.pdf"
EMBEDDER = "siglip"

rag = MultimodalRAG(pdf_file=PDF_FILE, embedder=create_embedder(EMBEDDER))
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini")) # For Ragas evaluation

FAISS Input Embeddings Shape: (11, 768)


### 3. Generate Dataset

In [3]:
# Check if QA file already exists
qa_filepath = "QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

if os.path.exists(qa_filepath):
    qa_path = qa_filepath
    print(f"Using existing QA file: {qa_path}")
else:
    qa_path = generate_qa_for_pdf(PDF_FILE)
    print(f"Generated new QA file: {qa_path}")

with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)
    

Using existing QA file: QA_subset_riksbanken.json


### 4. Answering the QA

In [4]:
with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)
    
# Generate dataset
rag_generated_answers = []

# Check if generated answers file already exists
rag_answers_path = "rag_generated_answers_" + os.path.basename(qa_path)

if os.path.exists(rag_answers_path):
    rag_generated_answers = json.load(open(rag_answers_path, 'r', encoding='utf-8'))
    print(f"Using existing RAG generated answers file: {rag_answers_path}")
else:
    for qa in qa_data:
        query = qa["question"]
        reference = qa["answer"]

        relevant_docs = rag.get_most_relevant_docs(query)
        response = rag.generate_answer(query, relevant_docs)
        rag_generated_answers.append(
            {
                "query":query,
                "retrieved_contexts":relevant_docs,
                "generated_answer":response,
                "true_answer":reference
            }
        )

    # Save the dataset to a JSON file
    output_dataset_file = "rag_generated_answers_" + os.path.basename(qa_path)

    with open(output_dataset_file, 'w', encoding='utf-8') as f:
        json.dump(rag_generated_answers, f, ensure_ascii=False, indent=4)
    print(f"Generated new RAG generated answers file: {output_dataset_file}")

Using existing RAG generated answers file: rag_generated_answers_QA_subset_riksbanken.json


### Print Generated Answers

In [5]:
for i, entry in enumerate(rag_generated_answers):
    print(f"Entry {i+1}:")
    print(f"x  Query: {entry['query']}")
    print(f"x  Retrieved Contexts: {entry['retrieved_contexts']}")
    print(f"x  Generated Answer: {entry['generated_answer']}")
    print(f"x  True Answer: {entry['true_answer']}")
    print("-" * 40)

Entry 1:
x  Query: What was the total amount of interest-bearing debt for Swedish non-financial companies at the end of 2023, and how is this amount divided between loans from banks and other lenders and issued debt securities?
x  Retrieved Contexts: [{'type': 'text', 'content': 'The real economy’s need for financial services \n23 unrated companies has also increased significantly. One contributing factor may be the low interest rates that followed the financial crisis, which lowered absolute re-turns and increased investor demand for riskier assets, as well as making the price of securities financing more favourable compared with the price of bank loans.  It is mainly foreign actors who invest in the debt securities issued by companies, fol-lowed by Swedish funds and Swedish insurance and pension companies, see Figure 12. The Riksbank owned 0.4 percent of the securities at the end of 2023, having pur-chased corporate bonds and commercial papers between September 2020 and De-cember 202

### Evaluate Retrieval

In [6]:
all_real_pages, all_retrieved_pages = [], []

for rag_answer in rag_generated_answers:
    real_page = next(
        item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
    )
    retrieved_pages = [doc["page_number"] for doc in rag_answer["retrieved_contexts"]]
    
    all_real_pages.append(real_page)
    all_retrieved_pages.append(retrieved_pages)

for k in [3, 5]:
    mrr = compute_mrr_at_k(all_retrieved_pages, all_real_pages, k)
    print(f"MRR@{k}: {mrr}")

MRR@3: 0.36666666666666664
MRR@5: 0.44666666666666666


### Evaluate Generation

In [ ]:
# faithfulness_and_relevance = evaluate_generation(rag_generated_answers, evaluator_llm)
# print(faithfulness_and_relevance)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'faithful_rate': 1.0000, 'relevance_rate': 1.0000}
